In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_PROJECT_ID = os.getenv("LANGCHAIN_PROJECT_ID")

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

In [3]:
# OpenAI 객체를 생성합니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

In [11]:
# 원하는 데이터 구조를 정의합니다.
class Topic(BaseModel): 
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [12]:
# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)

In [6]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [7]:
chain = prompt | model | parser  # 체인을 구성합니다.

In [13]:
# 질의 작성
question = "ai와 융합했을 때 오랫동안 변하지 않는 높은 가치를 가지는 분야가 뭐야. 가장 중요한 두가지를 말해."
answer = chain.invoke({"question":question})
answer

{'description': '의료 분야는 AI와 융합했을 때 오랫동안 변하지 않는 높은 가치를 가질 수 있습니다. AI는 진단 정확도를 높이고, 치료 계획을 최적화하며, 환자 모니터링을 개선하는 데 기여할 수 있습니다.',
 'hashtags': '#의료 #AI융합 #미래가치'}